In [2]:
from SPARQLWrapper import SPARQLWrapper, JSON
from datetime import datetime
import pandas as pd
import numpy as np

connect to db

In [2]:
sparql = SPARQLWrapper('https://dati.camera.it/sparql')

Create dataframe of the deputati from a legislation

In [3]:
leg = 'xviii'
year = '2021'

In [4]:
dep = pd.read_csv(f'data/dep_{leg}.csv')[['persona', 'nomeGruppo', 'nome', 'cognome']]
a = dep['cognome']+ ' ' + dep['nome']
dep['Parlamentare'] = a
dep.drop(['nome', 'cognome'], axis = 1, inplace= True)

a = []
for i in range(dep.shape[0]):
    a.append(dep['persona'][i][39:])

dep['id'] = a
dep.drop('persona', inplace= True, axis = 1)

p = []
dat = []
for i in dep['nomeGruppo']:
    e = i.split()
    p.append(e[-2][1:-1])
    d = e[-1].replace(')', '')
    dat.append(datetime.strptime(d[-10:], '%d.%m.%Y'))

dep['Partito'] = p
dep['dataIngresso'] = dat
dep.drop('nomeGruppo', axis = 1, inplace= True)

first_df = pd.DataFrame(columns= dep.columns)

for i in np.unique(dep['id']):
    if len(dep[dep['id'] == i]) < 2:
        first_df = pd.concat([first_df, dep[dep['id'] == i]])
    else:
        duplicate = dep[dep['id'] == i]
        first_df = pd.concat([first_df, duplicate.head(1)])


In [5]:
sparql.setReturnFormat(JSON)

final_df = first_df.copy()

# deputati of a specific legislation
if (year == '2013' and leg == 'xvii'):
   startmonth = 4
elif (year == '2018' and leg == 'xviii'):
   startmonth = 6
else: 
   startmonth = 1

if (year == '2018' and leg == 'xvii'):
   endmonth = 6
elif (year == '2022' and leg == 'xviii'):
   endmonth = 10
else: 
   endmonth = 13

for m in range(startmonth,endmonth):
   print(f'{m:02d}')
   if (m == 4 and year == '2013' and leg == 'xvii'):
      startday = 28
   else:
      startday = 1

   if (m == 10 and year == '2022' and leg == 'xviii'):
      endday = 22
   else:
      endday = 32
   print(startday,endday)
   for d in range(startday,endday):
      month = f'{m:02d}'
      day = f'{d:02d}'
      sparql.setQuery("""
      select distinct ?votazione ?data ?titolo ?numeroVotazione ?cognome ?nome ?espressione ?infoAssenza ?deputato where {
      ?votazione a ocd:votazione;
         dc:date '"""+year+month+day+"""';
         dc:date ?data;
         rdfs:label ?titolo;
         dc:description ?descrizione;
         dc:identifier ?numeroVotazione.


      ## voti espressi
      ?voto a ocd:voto;
         ocd:rif_votazione ?votazione;
         dc:type ?espressione;
         ocd:rif_deputato ?deputato.
      OPTIONAL{?voto dc:description ?infoAssenza}


      ## info deputato
      ?deputato foaf:surname ?cognome; foaf:firstName ?nome  


      } ORDER BY ?cognome
         
      """
      )
      try:
         ret = sparql.queryAndConvert()
         res = ret["results"]["bindings"]
      except Exception as e:
         print(e)

      dep = []
      vot = []
      numero = []
      for i in res:
         dep.append((i.get('deputato')).get('value')[40:46])
         vot.append((i.get('espressione')).get('value'))
         numero.append((i.get('numeroVotazione')).get('value'))

      votazione = pd.DataFrame([dep,vot, numero], index = ['id', 'Voto1', 'numero']).T
         
      for i in list(set(votazione['numero'])):
         espressione = votazione[votazione['numero'] == i][['id', 'Voto1']]
         espressione.columns= ('id', i)
         final_df = pd.merge(final_df, espressione, how = 'left')
            
print(final_df.shape)     

01
1 32
02
1 32
03
1 32
04
1 32
05
1 32
06
1 32
07
1 32
08
1 32
09
1 32
10
1 32
11
1 32
12
1 32
(660, 1749)


In [6]:
nan_count = final_df.isnull().sum(axis=1)
l = len(final_df.columns[4:])
final_df = final_df.drop(nan_count[nan_count >= l].index, axis = 0)

final_df.shape


dizionario_espressioni = {'Favorevole': np.float64(1), 'Non ha votato': np.float64(0), 'Contrario': np.float64(3), 'Astensione': np.float64(4)}
for i in final_df.columns[4:]:
    final_df[i] = final_df[i].map(dizionario_espressioni)

final_df = final_df.replace(np.nan, 0)

final_df.to_csv('./'+leg+'-years'+'/'+leg+'-'+year+'.csv', index=False)

In [7]:
final_df

,Parlamentare,id,Partito,dataIngresso,449002,449004,449001,449005,449003,450001,...,607021,608006,608002,608001,608005,608008,608004,608007,608009,608003
0,BASINI GIUSEPPE,200049,LEGA,2018-03-27,1.0,1.0,1.0,1.0,1.0,4.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,BERSANI PIER LUIGI,300026,MISTO,2018-03-23,3.0,1.0,3.0,1.0,1.0,1.0,...,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,4.0,1.0
2,LETTA ENRICO,300127,PD,2018-03-27,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,MINNITI DOMENICO,300146,PD,2018-03-27,3.0,1.0,3.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,FRANCESCHINI DARIO,300246,PD,2018-03-27,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
636,AVOSSA EVA,308521,PD,2018-03-27,0.0,0.0,0.0,0.0,0.0,0.0,...,3.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,3.0,1.0
637,CIAGA' GRAZIELLA LEYLA,308541,PD,2018-03-27,0.0,0.0,0.0,0.0,0.0,0.0,...,3.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,3.0,1.0
638,TIMBRO MARIA FLAVIA,308561,-SI,2018-04-10,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,4.0,1.0
639,CASU ANDREA,308600,PD,2018-03-27,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Merge

In [4]:
'''
leg = 'xvii'

year = 2013
merged = pd.read_csv(f'./{leg}-years/{leg}-{year}.csv')
print(merged.shape)

for year in [2014,2015,2016,2017,2018]:
    y = pd.read_csv(f'./{leg}-years/{leg}-{year}.csv')
    merged = pd.merge(merged, y, how = 'left')
    print(merged.shape)

merged = merged.replace(np.nan, 0)

merged.to_csv('./'+leg+'-years'+'/'+leg+'-merged.csv', index=False)
'''


(596, 2822)
(596, 7068)
(596, 14592)
(596, 19999)
(596, 24923)
(596, 24938)
